# Multi-Resource Scheduling

Tasks require multiple resources at the same time. We model each
resource with a cumulative function.


## Problem Description

Tasks require multiple resources at the same time (workers, machines, budget).
Each resource has a capacity and each task has a demand.

Objective: minimize makespan.


## Mathematical Model

Definitions:
$$
I_i: \text{task interval}, \quad d_{i,r}: \text{demand}, \quad C_r: \text{capacity}
$$

Constraints:
$$
\sum_i d_{i,r} \cdot active_i(t) \le C_r \quad \forall r, \forall t
$$

Objective:
$$
\min \max_i end_i
$$


## Imports


In [1]:
# Imports
from pycsp3 import *
from pycsp3_scheduling import *  

## Problem Data


In [2]:
resource_capacities = {
    'workers': 5,
    'machines': 2,
    'budget': 100,
}

multi_resource_tasks = [
    ('build', 10, 3, 1, 50),
    ('test', 5, 2, 1, 20),
    ('deploy', 3, 1, 0, 30),
    ('review', 4, 2, 0, 10),
    ('analyze', 6, 1, 1, 40),
]

horizon = sum(d for _, d, *_ in multi_resource_tasks)


## Build the Model


In [3]:
clear()
from pycsp3_scheduling.variables.interval import clear_interval_registry
clear_interval_registry()

intervals = {}
for name, duration, workers, machines, budget in multi_resource_tasks:
    intervals[name] = IntervalVar(
        start=(0, horizon),
        end=(0, horizon),
        size=duration,
        name=name
    )


In [4]:
worker_usage = None
machine_usage = None
budget_usage = None

for name, duration, workers, machines, budget in multi_resource_tasks:
    iv = intervals[name]
    if workers > 0:
        p = pulse(iv, workers)
        worker_usage = p if worker_usage is None else worker_usage + p
    if machines > 0:
        p = pulse(iv, machines)
        machine_usage = p if machine_usage is None else machine_usage + p
    if budget > 0:
        p = pulse(iv, budget)
        budget_usage = p if budget_usage is None else budget_usage + p

satisfy(cumul_range(worker_usage, 0, resource_capacities['workers']))
satisfy(cumul_range(machine_usage, 0, resource_capacities['machines']))
satisfy(cumul_range(budget_usage, 0, resource_capacities['budget']))


AssertionError: non authorized type 0 <= CumulFunction(pulse(build, 3) + pulse(test, 2) + pulse(deploy, 1) + pulse(review, 2) + pulse(analyze, 1)) <= 5 <class 'pycsp3_scheduling.functions.cumul_functions.CumulConstraint'>

In [ ]:
objective_expr = Maximum(end_time(intervals[name]) for name in intervals)
minimize(objective_expr)


## Solve


In [ ]:
result = solve()

if result in (SAT, OPTIMUM):
    print("Solution found." + (" (Optimal)" if result == OPTIMUM else ""))
    for name in intervals:
        val = interval_value(intervals[name])
        print(f"{name}: [{val.start}, {val.end})")
    makespan = max(interval_value(intervals[name]).end for name in intervals)
    print(f"Makespan: {makespan}")
else:
    print("No solution found.")


## Stats


In [ ]:
print("Model statistics:", model_statistics())
if result in (SAT, OPTIMUM):
    objective_value = globals().get("makespan")
    print("Solution statistics:", solution_statistics(status=result, objective=objective_value))


## Visualization


In [ ]:
if result in (SAT, OPTIMUM):
    visu.reset()
    makespan = max(interval_value(intervals[name]).end for name in intervals)
    visu.timeline("Multi-Resource Schedule", origin=0, horizon=makespan)
    visu.panel("Tasks")

    schedule = []
    for idx, name in enumerate(intervals):
        val = interval_value(intervals[name])
        schedule.append((val.start, val.end, name, idx))
    schedule.sort()

    for start, end, name, idx in schedule:
        visu.interval(start, end, name, color=idx)

    if visu.is_visu_enabled():
        visu.show()
    else:
        print("Visualization disabled (matplotlib not available).")
